# Ensemble methods

Ruixuan Dong

---

## Table of Contents
 - [Introduction](#0)
 - [Model averaging](#1)
     - [Stacking](#11)
     - [Super learning](#12)
 - [Simulated Dataset](#2)

<a name='0'></a>

## Introduction

In this section, we discuss ensemble methods. These are, loosely speaking, approaches for combining many learners to create a more powerful committee. Similarly, we can think of boosting as an ensemble method as well. In this section, we will focus on two approaches for ensemble learning: stacking and the so-called superlearner. Arguably, these are both just special cases of model averaging.

While some ideas in this section may seem simple, these approaches are extremely powerful and often used in practice.

<a name='1'></a>

## 1 Model averaging

To begin our discussion of model averaging, let us recall the bagging (bootstrap aggregated) estimator we discussed earlier in the course. To fit the bagging estimator, we constructed B independent bootstrapped datasets from our full training data \\({(y_i, x_i)}_{i=1}^n\\). Then, we fit a prediction model to each of the bootstrapped datasets. Let \\(\hat{f}_b(x)\\) denote the prediction based on the model from the bth bootstrapped dataset. The bagging estimator was then defined as simply the average of B different prediction models \\(\hat{f}_b\\), that is:

\\[ \hat{f}_{\text{bag}}(x) = \frac{1}{B} \sum_{b=1}^{B} \hat{f}_b(x) \\]

The utility of bagging, we argued, was that it could reduce the variance by virtue of averaging.

We can alternatively think of \\(\hat{f}_{\text{bag}}\\) as a model averaging approach: after all, it is defined as simply the average of $B$ different prediction models (if we ignore that each is fit to a different bootstrapped dataset). From this perspective, we can view model aggregation as a slightly different problem: given models \\(\{f_1, ..., f_B\}\\), how can we combine them in order to achieve the best combination (in the sense that it achieves the best prediction accuracy)? Taking their average is one approach, but perhaps a weighted average may perform better. Stacking is one approach to achieve an aggregated model based on a weighted average.

Let \\(\hat{f}_1(x), \ldots, \hat{f}_M(x)\\) be predictions coming from M distinct fitted models. Given these predictions, under squared-error loss, we seek weights \\(w = (w_1, \ldots, w_M)\\) such that:

\\[ \hat{w} =  \underset{w}{\text{arg min}}\ \mathbb{E}\left[\left(Y - \sum_{m=1}^M w_m \hat{f}_m(x)\right)^2\right] \\]

where the expectation is taken with respect to the distribution of $Y$ given the predictors. With \\(\hat{F}(x) \equiv [\hat{f}_1(x), \ldots, \hat{f}_M(x)]\\), it is straightforward to verify that:

\\[ \hat{w} = \mathbb{E}\left[\hat{F}(x)\hat{F}(x)'\right]^{-1} \mathbb{E}\left[\hat{F}(x)Y\right]. \\]

Given this \\(\hat{w}\\), one can show that:

\\[ \mathbb{E}\left[\left(Y - \sum_{m=1}^M \hat{w}_m \hat{f}_m(x)\right)^2\right] \leq \mathbb{E}\left[\left(Y - \hat{f}_m(x)\right)^2\right], \, \forall m, \\]

meaning that at the population level, the weighted average model \\(\sum_{m=1}^M \hat{w}_m \hat{f}_m(x)\\) is never worse than any individual model \\(\hat{f}_m\\) in terms of squared error.

Of course, while this is nice in theory, such an estimator cannot be put to use in practice. Replacing the expectations above with the sample counterparts often leads to poor prediction. For example, consider the case that \\(\hat{f}_m(x)\\) denotes the best subset regression model of size m. Then, using the sample version of this approach would also put all weight on \\(\hat{f}_M(x)\\), the largest best subsets model (i.e., \\(\hat{w}_M = 1\\) and \\(\hat{w}_m = 0\\) for all $m \neq M$). This is intuitive since this approach only accounts for model fit, not model complexity (as \\(\hat{f}_M\\) is the most complex model).



<a name='11'></a>

### 1.1 Stacking
One approach for model combination which avoids this issue is stacking or stacked generalizations. Let \\(\hat{f}^{-i}_m(x)\\) be the prediction at $x$ using model $m$, applied to a dataset with the $i$th sample removed. The stacked estimate of the weights $w$ is then defined as:

\\[ \hat{w}_{\text{st}} = \underset{w}{\text{arg min}} \sum_{i=1}^{n} \left( y_i - \sum_{m=1}^{M} w_m f^{-i}_m(x_i) \right)^2 \\]

so that the stacked estimate at x is \\(\sum_{m=1}^{M} \hat{w}^{\text{st}}_m \hat{f}_m(x)\\) where \\(\hat{f}_m\\) is the $m$th model fit to the entire dataset. This approach can be improved by imposing a non-negativity and sum-to-one constraint on the $w$

\\[ \tilde{w}_{\text{st}} = \underset{w}{\text{argmin}} \left( \sum_{i=1}^{n} \left( y_i - \sum_{m=1}^{M} w_m f_{m}^{-i}(x_i) \right)^2 \right), \ \sum_{m=1}^{M} w_m = 1, w_m \geq 0 \, \forall m. \\]

By using cross-validation preidctions $\hat{f}_m^{-i}(x_i)$, stacking avoids giving unfairly high weight to models with higher complexity.

This approach is more general than described here. For example, we can replace squared error with an alternative loss; or could modify the criterion so that the weight estimation criterion also depends on x, the point at which a new prediction is ultimately desired. One could, in principle, assign more weight to models where points near $x$ predict well.





<a name='12'></a>

### 1.2 Super learning

One approach for model combination, which is a popular variation of generalized stacking, is super learning. The prediction model output at the end of the procedure is the so-called super learner. This section will follow closely from Van der Laan et al. (2007). This method has achieved widespread use due to some nice theoretical results and its well-documented, easy-to-use software.

To formalize our discussion, suppose we have observed n realizations of the random pair \\((Y_i, X_i)\\). The goal is to estimate \\(E(Y | X) \equiv f^*(X)\\). For a particular dataset, suppose we have a library of estimators of \\(f^*\\). These estimators can be simple (e.g., a stump or a linear regression model) or quite complex (e.g., a random forest or boosted tree). The key is that each \\(\hat{f}\\) is a function mapping the training data to a prediction.

Let L denote the library of estimators and let K(n) denote the cardinality of L. The super learning algorithm proceeds as follows:

- **1**: Fit each estimator in L to the entire training dataset to obtain predictions \\(\hat{f}_1(x), \ldots, \hat{f}_{K(n)}(x)\\).
- **2**: Split the training data into a training and validation sample according to a V-fold cross-validation scheme.
- **3**: For each fold, fit each algorithm in L on the training data, and store the predictions on the corresponding validation data.
- **4**: Stack the predictions from each estimator together to create an n × K(n) matrix of predictions.
- **5**: Propose a family of weighted combinations of the candidate estimators indexed by a weight vector.
- **6**: Determine the weight vector that minimizes the cross-validated risk of the candidate estimator.
- **7**: Output the Super Learner, which is a weighted combination of the fitted models.


<a name='2'></a>
### 2 Simulated Dataset

In this part, we use the dataset generated by Synthetic SCG data generator https://github.com/wsonguga/DataDemo. In the data file, each row includes sensor data (10 seconds * 100Hz) + HeartRate + RespiratoryRate + SystolicBloodPressure + DiastolicBloodPressure.

Now we would like to give a introduction about the synthetic SCG dataset we generated. 

Based on Synthetic SCG data generator, we generated an artificial (synthetic) scg signal of a given duration (10 seconds, i.e. duration=10) and sampling rate (100Hz, i.e. sampling rate=100) using a model based on Daubechies wavelets to roughly approximate cardiac cycles.

Besides, we set 
 - heart rate to be randomly chosen from the intgers range from 50 to 150, with the desired heart rate standard deviation (beats per minute) equal to 1.
 - respiratory rate to be randomly chosen from the intgers range from 10 to 30
 - diastolic blood pressure to be randomly chosen from the intgers range from 60 to 99
 - systolic blood pressure to be randomly chosen from the intgers range from 100 to 160

The sample size of the current dataset is 6,000 in total.


**Problem Statement:** The generated dataset containing: 
- a dataset set ("lower.csv") of 3,000 samples labeled as lower (100<=systolic blood pressure<140) 
- a dataset set ("higher.csv") of 3,000 samples labeled as higher (140<=systolic blood pressure<=160) 
- each sample is of shape (1, 1003) where 1003 is for the 1000-d signal and heart rate, respiratory rate and diastolic blood pressure

In this part, we will build a simple kNN classifier that can correctly classify samples as lower or higher (SBP).

Let's get more familiar with the dataset. Load the data by running the following code.

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [9]:
column_names = [str(i) for i in range(1, 1001)] + ['heart_rate', 'respiratory_rate', 'systolic', 'diastolic']
total = pd.read_csv('total_large.csv', 
                     header=None, 
                     names=column_names)
total.head(3)

,1,2,3,4,5,6,7,8,9,10,...,995,996,997,998,999,1000,heart_rate,respiratory_rate,systolic,diastolic
0,5.439234e-08,2.583753e-07,0.000001,0.000004,0.000008,0.000006,-4.897503e-07,-0.000004,-2.029141e-07,3.029687e-06,...,-3.750468e-08,-3.504179e-08,-3.266654e-08,-2.969555e-08,-2.688206e-08,-2.599564e-08,109.0,19.0,160.0,66.0
1,5.781177e-08,3.850786e-07,0.000002,0.000007,0.000007,-0.000001,-3.642447e-06,0.000002,8.308896e-07,-1.850758e-06,...,-3.937486e-08,-3.615418e-08,-3.250324e-08,-2.930146e-08,-2.813366e-08,-2.915194e-08,131.0,15.0,153.0,64.0
2,3.434446e-08,2.098668e-07,0.000003,0.000006,-0.000003,0.000002,-1.939304e-06,0.000001,-9.990558e-07,3.452373e-07,...,-3.199401e-08,-2.472291e-08,-1.890941e-08,-1.882332e-08,-2.188260e-08,-2.335538e-08,128.0,14.0,120.0,85.0


In [10]:
def signal2matrix(total):
    total = total.values

    numberOfLines = len(total)
    returnMat = np.zeros((numberOfLines, 1003))
    classLabelVector = []
    index = 0

    for line in total:
        returnMat[index, :1002] = line[:1002]
        returnMat[index, 1002] = line[1003]
        if 100 <=line[1002]< 140:
            classLabelVector.append(1)
        elif 140 <=line[1002]<= 160:
            classLabelVector.append(2)
        index += 1
    return returnMat, classLabelVector

def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - np.tile(minVals, (m, 1))
    normDataSet = normDataSet / np.tile(ranges, (m, 1))
    return normDataSet, ranges, minVals

In [11]:
feature_matrix, class_labels = signal2matrix(total)
norm_feature_matrix, ranges, minVals = autoNorm(feature_matrix)
np.shape(feature_matrix)
np.shape(class_labels)

(15000,)

In [ ]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, 
                                                    class_labels, 
                                                    test_size=0.3, 
                                                    random_state=42)

### Using Stacking Method

In [17]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

base_learners = [
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('svm', SVC(random_state=42))
]

stacking_clf = StackingClassifier(
    estimators=base_learners, 
    final_estimator=LogisticRegression(),
    cv=5
)

stacking_clf.fit(X_train, y_train)

y_pred_stack = stacking_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_stack))
print(classification_report(y_test, y_pred_stack))

0.9751111111111112
              precision    recall  f1-score   support

           1       0.98      0.98      0.98      2923
           2       0.97      0.96      0.96      1577

    accuracy                           0.98      4500
   macro avg       0.97      0.97      0.97      4500
weighted avg       0.98      0.98      0.98      4500



### Using Super learning method

In [19]:
from sklearn.ensemble import VotingClassifier

classifiers = [
    ('lr', LogisticRegression(random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('svm', SVC(probability=True, random_state=42))
]

voting_clf = VotingClassifier(estimators=classifiers, voting='soft')

voting_clf.fit(X_train, y_train)

y_pred_voting = voting_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_voting))
print(classification_report(y_test, y_pred_voting))



0.9751111111111112
              precision    recall  f1-score   support

           1       0.98      0.98      0.98      2923
           2       0.97      0.96      0.96      1577

    accuracy                           0.98      4500
   macro avg       0.97      0.97      0.97      4500
weighted avg       0.98      0.98      0.98      4500



---

## References

- Mark J Van der Laan and Sherri Rose. _Targeted learning: causal inference for observational and experimental data._ Springer Science & Business Media, 2011.
- Mark J Van der Laan, Eric C Polley, and Alan E Hubbard. _Super learner._ Statistical applications in genetics and molecular biology, 6(1), 2007.